In [5]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
from utils import load_train_test,make_plot,RMSE,mask_check
import json
import matplotlib.pyplot as plt
import seaborn as sns



In [6]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = ["quantile","diffuse","logistic"]
#missingtypelist = ["diffuse"]

In [11]:
def mean_imp(missingtypelist,datalist_name,seed = 1,nfold = 5):


    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        print(missingtype)
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")



        for dataname in datalist:
            print(dataname)
            path = f"../impute/{missingtype}/{dataname}/one"
            path = f"../impute/{missingtype}/{dataname}/zero"
            path = f"../impute/{missingtype}/{dataname}/random"
            if not os.path.exists(path):
                # If the path does not exist, create it
                os.makedirs(path)

            for rule_name in missing_rule:

                    directory_path = f"../datasets/{dataname}"  
                    # Opening JSON file
                    f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
                    index_file = json.load(f)

                    for fold in index_file:

                        train_values,train_masks,test_values,test_masks = load_train_test(index_file[fold],missingtype,rule_name,directory_path,dataname)

                        train_values_na = np.where(train_masks == 0, np.nan, train_values)
                        test_values_na = np.where(test_masks == 0, np.nan, test_values)
                        
                        #imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=1)
                        imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
                        imp = RandomImputer()
                        #imp_random.fit(train_values_na)
                        test_imp = imp.fit_transform(test_values_na)
                        train_imp = imp.fit_transform(train_values_na)

                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_train.npy', train_imp.astype("float32"))
                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_test.npy', test_imp.astype("float32"))

                        if  (test_imp.shape != test_values_na.shape):
                            print("test shape")
                        if  (train_imp.shape != train_values_na.shape):
                            print("train shape")


            

In [12]:
real_datalist = [
            "banknote",
             "concrete_compression",
             "wine_quality_white",
            "wine_quality_red",
            "california",
            "climate_model_crashes",
            "connectionist_bench_sonar",
            "qsar_biodegradation", 
            "yeast", 
            "yacht_hydrodynamics"
            ]
missingtypelist = ["quantile",
                   "diffuse",
                   "logistic"
                   ]

mean_imp(missingtypelist,"real")

quantile
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
diffuse
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
logistic
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics


In [2]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted, check_array

class RandomImputer(BaseEstimator, TransformerMixin):
    """
    An imputer transformer that fills missing values with random floats in the range [0, 1].
    """

    def __init__(self, missing_values=np.nan):
        self.missing_values = missing_values

    def fit(self, X, y=None):
        # Check that X is in the right format
        X = check_array(X, force_all_finite='allow-nan')
        self.is_fitted_ = True
        return self

    def transform(self, X):
        # Check if fit had been called
        check_is_fitted(self, 'is_fitted_')
        # Check that X is in the right format
        X = check_array(X, force_all_finite='allow-nan')

        # Replace missing values with a random float between 0 and 1
        if np.isscalar(self.missing_values):
            mask = np.isnan(X) if np.isnan(self.missing_values) else X == self.missing_values
        else:
            mask = np.isin(X, self.missing_values)
        random_values = np.random.rand(*X.shape)
        X[mask] = random_values[mask]

        return X
